Fine-tune model

In [ ]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import Trainer, TrainingArguments
from transformers import EarlyStoppingCallback


data = pd.read_csv(r'data\fine-tuning\keyword_label.csv')

data['label'] = data['label'].astype('category').cat.codes

train_texts, val_texts, train_labels, val_labels = train_test_split(
    data['keyword'].tolist(),
    data['label'].tolist(),
    test_size=0.2,
    random_state=42
)

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)

class KeywordDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = KeywordDataset(train_encodings, train_labels)
val_dataset = KeywordDataset(val_encodings, val_labels)

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(data['label'].unique()))

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=5,  
    per_device_train_batch_size=16,  
    per_device_eval_batch_size=64,
    warmup_steps=1000,  
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,  
    evaluation_strategy="steps",  
    save_total_limit=2,  
    load_best_model_at_end=True,  
    metric_for_best_model="eval_loss",  
    greater_is_better=False,  
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=4)]
)

trainer.train()

model_save_path = r'fine_tune_bert'
model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)

print(f"Model saved to {model_save_path}")


Start embedding

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
import pandas as pd
import time

start_time = time.time()

model_name = r'\fine_tuned_model\fine_tuned_bert'

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name
)


df = pd.read_csv(r'data.csv')

embedding_text = []
line = 0
print(df)
for item in df['keyword']:
    line+=1
    print(line)

    inputs = tokenizer(item, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs, output_hidden_states=True)
    event_embedding = outputs.hidden_states[-1].mean(dim=1).cpu().numpy().flatten()
    embedding_text.append(event_embedding)

data = pd.DataFrame(embedding_text)
data['timestamp'] = df['date']
data.to_csv(r'BERT_EmbeddingText.csv', index=False)

end_time = time.time()
execution_time = end_time - start_time
print(f"Execution time: {execution_time} seconds")
